# [Internal] Luminary Cloud SDK Tutorial
**This notebook is intended for internal use only. For a customer facing example
notebook, see: `python/sdk/docs/source/assets/luminarycloud-quickstart/notebooks/quickstart-notebook.ipynb`**

### Log in for the first time

If you haven’t yet opened up the Luminary web app, you will need to log in and accept the terms and conditions before proceeding.


### Import the SDK and Setup the Client

**To run this notebook against a different environment, update `api_env_name` in
the block below.**


In [ ]:
import logging, os

logger = logging.getLogger("luminarycloud")
logger.setLevel(logging.INFO)

import luminarycloud as lc
from luminarycloud.enum import QuantityType, ReferenceValuesType
from luminarycloud.reference_values import ReferenceValues
import pandas as pd
import sys

sys.path.insert(0, "..")
from sdk_util import get_client_for_env


# <internal>
# Please keep the <internal> tags because assistant will use it to filter out the internal code.
# NOTE: update api_env_name to make API calls against a different environment
api_env_name = "main"
lc.set_default_client(get_client_for_env(env_name=api_env_name))
print(f"SDK is connected to env: {api_env_name}")
# </internal>

print(f"SDK version: {lc.__version__}")
print(f"SDK installation location: {lc.__file__}")

### List projects and get project

In [ ]:
projects = lc.list_projects()

if len(projects) > 0:
    print("All projects:")
    [print(f"- {p.id} {p.name}") for p in projects]
    project = projects[0]
else:
    print("You have no projects. Creating one...")
    import datetime

    project = lc.create_project(name=f"Project {datetime.datetime.now()}")

print(f"\nUsing project: {project.id} {project.name}")

### List Meshes / Upload a new mesh

In [ ]:
meshes = project.list_meshes()
if len(meshes) > 0:
    print(f"All meshes in project {project.id}:")
    [print(f"- {m.id} {m.name}") for m in meshes]
    mesh = meshes[0]
else:
    # NOTE: you may need to update the path
    mesh = project.upload_mesh(
        "../docs/luminarycloud-quickstart/notebooks/data/naca0012_inv.lcmesh"
    )
    print(f"Uploaded mesh {mesh.id} {mesh.name}")


print(f"\nUsing mesh: {mesh.id} {mesh.name} {mesh.status}")
# mesh.delete()

### Create new simulation with uploaded mesh
Now lets create a simulation with the mesh we just uploaded. We'll use a fvm param json we downloaded from the UI.

> NOTE: to download the params from the UI, click `...` next to the `Upload` button in the main settings panel. In the dropdown, select `Download Fvm Param`.

In [ ]:
params_json_path = "../docs/luminarycloud-quickstart/notebooks/data/api_simulation_params.json"
simulation_template = project.create_simulation_template(
    name="SDK Sim Template", params_json_path=params_json_path
)

simulations = project.list_simulations()
if len(simulations) > 0:
    print(f"All simulations in project {project.id}:")
    [print(f"- {s.id} {s.name} {s.status.name}") for s in simulations]
    simulation = simulations[0]
else:
    # NOTE: you may need to update the path
    simulation = project.create_simulation(
        mesh_id=mesh.id,
        name="SDK Sim with Client Params JSON",
        simulation_template_id=simulation_template.id,
        batch_processing=False,
    )
    print(f"Created simulation: {simulation.id} {simulation.name}  {simulation.status.name}")

print(f"\nUsing simulation: {simulation.id} {simulation.name}  {simulation.status.name}")

The simulation is now running!

Let's wait for it to finish...

In [ ]:
status = simulation.wait(print_residuals=False, interval_seconds=10)
print("Simulation finished with status:", status.name)

### Post-processing

Now that the simulation is done, let's download and inspect the residuals...

In [ ]:
with simulation.download_global_residuals() as stream:
    # since this is a steady state simulation, we can drop these columns
    residuals_df = pd.read_csv(stream, index_col="Iteration index").drop(
        ["Time step", "Physical time"], axis=1
    )
residuals_df

Here we plot the residuals, similar to what we see in the UI.

In [ ]:
ax = residuals_df.plot(logy=True, figsize=(12, 8))

In addition to residuals, the SDK also support additional outputs (specified as part of the params when creating a simulation). Let download and inspect the Lift output...

**NOTE:** the surface names in the request below come from the mesh; we will soon
have a GetMeshMetadata call that will allow SDK users to see the mesh names
in order to pass the ones they want to this request.

In [ ]:
ref_vals = ReferenceValues(
    reference_value_type=ReferenceValuesType.PRESCRIBE_VALUES,
    area_ref=1.0,
    length_ref=1.0,
    p_ref=101325.0,
    t_ref=273.15,
    v_ref=265.05709547039106,
)

# see documentation for more details about optional parameters
with simulation.download_surface_output(
    QuantityType.LIFT, ["0/bound/airfoil"], frame_id="body_frame_id", reference_values=ref_vals
) as stream:
    # since this is a steady state simulation, we can drop these columns
    lift_df = pd.read_csv(stream, index_col="Iteration index").drop(
        ["Time step", "Physical time"], axis=1
    )
lift_df

In [ ]:
ref_vals = ReferenceValues(
    reference_value_type=ReferenceValuesType.PRESCRIBE_VALUES,
    area_ref=1.0,
    length_ref=1.0,
    p_ref=101325.0,
    t_ref=273.15,
    v_ref=265.05709547039106,
)

with simulation.download_surface_output(
    QuantityType.PRESSURE_FORCE,
    ["0/bound/airfoil"],
    frame_id="body_frame_id",
    reference_values=ref_vals,
    force_direction=lc.types.Vector3(1, 0, 0),
) as stream:
    # since this is a steady state simulation, we can drop these columns
    pressure_df = pd.read_csv(stream, index_col="Iteration index").drop(
        ["Time step", "Physical time"], axis=1
    )
pressure_df

In [ ]:
ax = lift_df.plot(figsize=(12, 8))

Lastly, we can download the surface and volume solutions for external post-processing with Paraview:

In [ ]:
latest_solution = simulation.list_solutions()[-1]
with latest_solution.download_surface_data() as streaming_tar_file:
    path = f"./surface_data_{simulation.id}"
    streaming_tar_file.extractall(path)
    print(f"Extracted files to {path}:")
    print("\t" + "\n\t".join(os.listdir(path)))

In [ ]:
with latest_solution.download_volume_data() as streaming_tar_file:
    path = f"./volume_solution_{simulation.id}"
    streaming_tar_file.extractall(path)
    print(f"Extracted files to {path}:")
    for root, dirs, filenames in os.walk(path):
        print("\t" + "\n\t".join([os.path.join(root, file) for file in filenames]))